In [29]:
import os
import paramiko
from netmiko import ConnectHandler
from datetime import datetime

In [30]:
#1. 장비리스트 생성
##출력값: 전체장비list(2중list)
#txt안에 있는 줄 수 만큼(장비수) 반복하여 장비별 list(각 장비별 한줄씩)를 생성하는 메서드

def get_all_device():
    device_file = "/root/jupyter/blue_network/data.txt"
    device_all = []
    f = open(device_file, "r")
    lines = f.readlines()

    for line in lines:
        parts = line.strip().split("_")  # 요소별로 분리됨
        device_all.append(parts)         # 그대로 리스트에 추가
    f.close()    
    return device_all

    
#2. 장비별 파일생성함수
#입력값: 파일내용-list , 경로-str
#파일 생성함수 폴더에 dns.txt안에 있는 줄 수 만큼 반복하여 파일을 생성하는 메서드 ###DNS의 경우
#장비관리파일.txt안에 있는 줄 수 만큼 반복하여 파일을 생성하는 메서드
#최종결과물: /경로/ww1_1.1.1.1_root_123456789_hj.dat

def gen_file(device_all):
    n = len(device_all)
    path = "/root/jupyter/blue_network/Machines/"
    
    #기기목록부터 최신화
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)

    for device in device_all:
        domain, ip, usr, pwd, master = device[:5]
        filename = f"{domain}_{ip}_{usr}_{pwd}_{master}.dat"
        with open(path + filename, "w") as f:
            f.close()
            

#3. 폴더 생성함수
#최종결과물 : 헤드폴더 - 팀원별 폴더
def gen_dir():
    
    # def cmdcall(command):
    #     result = os.popen(command).read().rstrip('\n')
    #     return result

    name = ["hw","hj","th","he","sh"]
    group = "blue_network"
    path = "/root" #cmdcall("pwd")

    #2. 입력결과 토대로 트리구조 생성
    ## 주의 : mkdir은 window와 unix에서만 가능
    head_path = path+ "/" + group
    os.makedirs(head_path, exist_ok=True)
    
    for i in name:
        branch_path = head_path +"/"+ i
        os.makedirs(branch_path, exist_ok=True)
    print("------------------RESULT----------------------")
    print(os.popen("ls "+ path +" -al").read())


def device_manage():
device_all = get_all_device()
gen_file(device_all)
print("Device updated successfully!")

In [32]:
# 핑 보내는 프로그램

import os
from datetime import datetime


def extractiplist(): # 모든 IP정보 및 접속 정보를 가져오는 함수
    files = os.popen("ls /root/blue_network/Machines").readlines()
    iplist = []

    for i in files:
        splitedData = i.split("_")
        iplist.append(splitedData[1])

    return iplist

def pingTest(iplist): # 핑을 보내는 함수
    data = ""
    errmachines = []
    path = "/root/blue_network/Machines/"
    for i in iplist:
        data = os.popen(f"ping -c 3 -w 1 {i}").read() # 핑을 3번 보내고 1초만 기다리라는 의미
        if "ttl" not in data: # 성공 메시지에 ttl이라는 값이 들어가 있음 그래서 이게 없으면 에러로 판별
            errmachines.append(i)
            fname = os.popen(f"ls {path} | grep '{i}'").read().strip()
            if fname:
                f = os.path.join(path,fname)
                now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                with open(f,"w") as errmsg:
                    errmsg.write(f"---{now}:::PING ERROR REPORT---\n")
                    errmsg.write(data)
                print(f"Saved Error Report for {fname}")
    return errmachines

def errorReport(errmachines): # 분석 결과 출력 함수
    print("감지된 에러 목록")
    for i in errmachines:
        print(i)
    print("------------------------------------------------")

def ping_all(): # 전체적인 ping 점검 및 에러 있는 ip 리스트 반환
    iplist = extractiplist()
    errmachines = pingTest(iplist)
    errorReport(errmachines)
    return errmachines # [ip1, ip2, ip3 ...]

if __name__ == "__main__":
    ping_all()

In [33]:
import os
import paramiko

# 설정 정보 확인 프로그램

def sshrun(user, pwd, ip, cmd): # 직접적인 명령어 처리
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy()) # ssh 서버가 주는 공개키를 자동으로 받으려고 하기 위해
    cli.connect(ip, username=user, password=pwd, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd) # 명령을 전달하는 메서드 (입력값, 출력값, 에러값)
    return stdout


def showconfigs(sshinfo, flag, machine): # 정보 출력 함수
    options = {"vlan" : "sh vlan-switch bri", "ip" : "sh ip int bri", "trunk" : "sh int trunk", "routing" : "sh ip route", "frame" : "sh frame-relay map", "vtp" : "sh vtp status", "run" : "sh run", "nat" : "sh run", "dhcp" : "sh ip dhcp binding"}

    for i in sshinfo:
        result = sshrun(i[1], i[2], i[0], options[flag]).read().decode("utf-8")

        if flag == "run": # sh run인 케이스 (모든 설정 출력 명령어)
            os.makedirs("/root/jupyter/blue_network/Machine_status", exist_ok=True)
            f = open(f"/root/jupyter/blue_network/Machine_status/run_{machine}_{i[0]}.txt", "w")
            f.write(result)
            f.close()
            print(f"run_{machine}_{i[0]} created!")
            
        elif flag == "nat": # nat인 경우 sh run을 통해서 전체 설정에서 ip nat pool 정보만 추출하도록 지정
            start = result.find("ip nat pool") # ip nat pool 정보의 시작 인덱스
            
            if start != -1: # 만약 nat pool 설정이 되어있는 경우
                end = result.find("!", start) # ip nat pool의 마지막 정보의 위치 인덱스    
                print(f"{machine} {i[0]}의 {flag}정보")
                print("*************************************************")
                print(result[start:end-1]) # ip nat pool 영역만 추출
                print("*************************************************\n")
            else:
                continue
        else: # 나머지
            print(f"{machine} {i[0]}의 {flag}정보")
            print("*************************************************")
            print(result)
            print("*************************************************\n")
            
if __name__ == "__main__":
    showconfigs(sshinfo, flag, machine)

In [ ]:
import os
from datetime import datetime
from netmiko import ConnectHandler, NetmikoTimeoutException, NetmikoAuthenticationException

# 장비 일괄 세팅하는 함수(현재는 mrtg기능만)

if __name__ == "__main__":

    def setconfigs(router_ips, switch_ips, flag): #설정값 입력 함수. ro/sw 따로관리
    ###1. 서비스 선택
    ###2. 서비스별 장비정보 입력
    ###3. 서비스별 장비 명령어 입력
    ###4. 전체반복적용
        devices = []
    
        #1. 서비스 선택
        if flag == "mrtg":  # mrtg인 케이스
    
            # 1. 기기별 snmp 설정정보 입력
            #### 1-1.라우터 정보 list
            for i in router_ips:
                devices.append({
                    'device_type': 'cisco_ios',
                    'host': i,
                    'username': 'bn',
                    'password': 'cisco',
                    'community': 'router'
                })
            ##### 1-2.스위치 정보 list
            for i in switch_ips:
                devices.append({
                    'device_type': 'cisco_ios',
                    'host': i,
                    'username': 'bn',
                    'password': 'cisco',
                    'community': 'switch'
                })
            # 2. 장비별 명령어 설정
            config_cmd = "snmp-server community " + device['community'] + " RO"
    
        else: #이상한 값을 넣었을 때
            print("잘못된 정보입니다.")
            retrun -1
            
        # 3. 전체 장비 대상 자동 설정 루프
        for device in devices:
            try:
                net_connect = ConnectHandler(**device)   # SSH 연결 시작
                output = net_connect.send_config_set([config_cmd])  # 장비별 설정 적용 (리스트 형태로 전달)
        
                print("--- " + device['host'] + " 설정 결과 ---")
                print(output)
        
                net_connect.disconnect()    # 연결 종료
    
            except (NetmikoTimeoutException, NetmikoAuthenticationException) as e:
                # SSH 연결 실패
                print(f"!!! {device['host']} SSH Connection Failed !!!")
                
                # 장비 파일위치
                path = "/root/blue_network/Machines/"
                ip = device['host'] # device 딕셔너리에 있는 IP 주소
                
                fname = os.popen(f"ls {path} | grep '{ip}'").read().strip()           
                if fname:
                    full_path = os.path.join(path, fname)
                    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    
                    # "a" 모드로 열어서 로그 누적 기록
                    with open(full_path, "a") as f:
                        f.write(f"\n------- {now} SSH Connection Failed -------\n")
                        f.write(f"Err: {str(e)}\n") # 에러 원인(타임아웃 등)
                        f.write("-------------------------------------------------\n")
                    
                    print(f"[Log] {fname} SSH Connection Failed")
                else:
                    print(f"[NOT FOUND] {ip} file Not Found")
        
            except Exception as other_e:
                # 예상치 못한 다른 에러 처리
                print(f"Unexpected Error: {other_e}")
    setconfigs(router_ips, switch_ips, flag)